# 대규모 언어 모델의 저순위 적응(LoRA)

<Tip warning={true}>

현재 LoRA는 `UNet2DConditionalModel`의 어텐션 레이어에만 지원됩니다. 또한 제한된 용량으로 LoRA를 사용하여 DreamBooth용 텍스트 인코더를 미세 조정하는 것도 지원합니다. DreamBooth용 텍스트 인코더를 미세 조정하면 일반적으로 더 나은 결과를 얻을 수 있지만 컴퓨팅 사용량이 증가할 수 있습니다.

</Tip>

[대규모 언어 모델의 저순위 적응(LoRA)](https://arxiv.org/abs/2106.09685)은 메모리를 덜 소비하면서 대규모 모델의 학습을 가속화하는 학습 방법입니다. 기존 가중치에 순위 분해 가중치 행렬 쌍( **업데이트 행렬**이라고 함)을 추가하고 새로 추가된 가중치만 학습합니다. 여기에는 몇 가지 장점이 있습니다.

- 이전 사전 학습된 가중치는 고정되어 모델이 [파국적 망각](https://www.pnas.org/doi/10.1073/pnas.1611835114)에 덜 취약합니다.
- 순위 분해 행렬은 원래 모델보다 매개변수가 훨씬 적으므로 학습된 LoRA 가중치를 쉽게 이식할 수 있습니다.
- LoRA 행렬은 일반적으로 원래 모델의 어텐션 레이어에 추가됩니다. 🧨 Diffusers는 [load_attn_procs()](https://huggingface.co/docs/diffusers/main/en/api/loaders#diffusers.loaders.UNet2DConditionLoadersMixin.load_attn_procs) 메서드를 제공하여 LoRA 가중치를 모델의 어텐션 레이어에 로드합니다. `scale` 매개변수를 통해 모델이 새 학습 이미지에 적응하는 정도를 제어할 수 있습니다.
- 메모리 효율성이 뛰어나 Tesla T4, RTX 3080 또는 RTX 2080 Ti와 같은 소비자 GPU에서 미세 조정을 실행할 수 있습니다! T4와 같은 GPU는 Kaggle 또는 Google Colab 노트북에서 무료로 쉽게 사용할 수 있습니다.

<Tip>

💡 LoRA는 어텐션 레이어에만 국한되지 않습니다. 저자들은 언어 모델의 어텐션 레이어를 수정하는 것만으로도 뛰어난 효율성으로 우수한 다운스트림 성능을 얻을 수 있다는 것을 발견했습니다. 이것이 모델의 어텐션 레이어에 LoRA 가중치를 추가하는 것이 일반적인 이유입니다. LoRA 작동 방식에 대한 자세한 내용은 [효율적인 Stable Diffusion 미세 조정을 위한 LoRA 사용](https://huggingface.co/blog/lora) 블로그를 확인하십시오!

</Tip>

[cloneofsimo](https://github.com/cloneofsimo)는 인기 있는 [lora](https://github.com/cloneofsimo/lora) GitHub 리포지토리에서 Stable Diffusion에 대한 LoRA 학습을 처음으로 시도했습니다. 🧨 Diffusers는 이제 [텍스트-이미지 생성](https://github.com/huggingface/diffusers/tree/main/examples/text_to_image#training-with-lora) 및 [DreamBooth](https://github.com/huggingface/diffusers/tree/main/examples/dreambooth#training-with-low-rank-adaptation-of-large-language-models-lora)에 대한 LoRA를 사용한 미세 조정을 지원합니다. 이 가이드에서는 두 가지 모두 수행하는 방법을 보여줍니다.

모델을 커뮤니티에 저장하거나 공유하려면 Hugging Face 계정에 로그인하십시오(아직 계정이 없다면 [여기](https://huggingface.co/docs/diffusers/main/en/training/hf.co/join)에서 만드십시오).

```bash
huggingface-cli login
```

## 텍스트-이미지 변환

수십억 개의 매개변수를 가진 Stable Diffusion과 같은 모델을 미세 조정하는 것은 느리고 어려울 수 있습니다. LoRA를 사용하면 확산 모델을 훨씬 쉽고 빠르게 미세 조정할 수 있습니다. 8비트 옵티마이저와 같은 트릭을 사용하지 않고도 11GB의 GPU RAM만으로 하드웨어에서 실행할 수 있습니다.

### 학습[[text-to-image-training]]

자신만의 포켓몬을 생성하기 위해 [Pokémon BLIP 캡션](https://huggingface.co/datasets/lambdalabs/pokemon-blip-captions) 데이터셋에서 [`stable-diffusion-v1-5`](https://huggingface.co/runwayml/stable-diffusion-v1-5)를 미세 조정해 보겠습니다.

`MODEL_NAME` 환경 변수(Hub 모델 리포지토리 ID 또는 모델 가중치가 포함된 디렉터리 경로)를 지정하고 [`pretrained_model_name_or_path`](https://huggingface.co/docs/diffusers/en/api/diffusion_pipeline#diffusers.DiffusionPipeline.from_pretrained.pretrained_model_name_or_path) 인수에 전달합니다. 또한 학습하려는 데이터셋의 이름으로 `DATASET_NAME` 환경 변수를 설정해야 합니다. 자신의 데이터셋을 사용하려면 [학습용 데이터셋 만들기](https://huggingface.co/docs/diffusers/main/en/training/create_dataset) 가이드를 참조하십시오.

`OUTPUT_DIR` 및 `HUB_MODEL_ID` 변수는 선택 사항이며 Hub에 모델을 저장할 위치를 지정합니다.

```bash
export MODEL_NAME="runwayml/stable-diffusion-v1-5"
export OUTPUT_DIR="/sddata/finetune/lora/pokemon"
export HUB_MODEL_ID="pokemon-lora"
export DATASET_NAME="lambdalabs/pokemon-blip-captions"
```

학습을 시작하기 전에 알아야 할 몇 가지 플래그가 있습니다.

* `--push_to_hub`는 학습된 LoRA 임베딩을 Hub에 저장합니다.
* `--report_to=wandb`는 학습 결과를 Weights & Biases 대시보드에 보고하고 기록합니다(예시로 이 [보고서](https://wandb.ai/pcuenq/text2image-fine-tune/runs/b4k1w0tn?workspace=user-pcuenq)를 참조하십시오).
* `--learning_rate=1e-04`, LoRA를 사용하면 일반적으로 사용하는 것보다 높은 학습률을 사용할 수 있습니다.

이제 학습을 시작할 준비가 되었습니다(전체 학습 스크립트는 [여기](https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/train_text_to_image_lora.py)에서 찾을 수 있습니다). 학습은 11GB RAM의 2080 Ti GPU에서 약 5시간이 걸리며 리포지토리에 모델 체크포인트와 `pytorch_lora_weights`를 만들고 저장합니다.

```bash
accelerate launch --mixed_precision="fp16"  train_text_to_image_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --dataloader_num_workers=8 \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=15000 \
  --learning_rate=1e-04 \
  --max_grad_norm=1 \
  --lr_scheduler="cosine" --lr_warmup_steps=0 \
  --output_dir=${OUTPUT_DIR} \
  --push_to_hub \
  --hub_model_id=${HUB_MODEL_ID} \
  --report_to=wandb \
  --checkpointing_steps=500 \
  --validation_prompt="A pokemon with blue eyes." \
  --seed=1337
```

### 추론[[text-to-image-inference]]

이제 [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)에서 기본 모델을 로드한 다음 [DPMSolverMultistepScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/multistep_dpm_solver#diffusers.DPMSolverMultistepScheduler)를 사용하여 모델을 추론에 사용할 수 있습니다.

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

model_base = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(model_base, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

미세 조정된 모델에서 LoRA 가중치를 기본 모델 가중치 *위에* 로드한 다음 추론 속도를 높이기 위해 파이프라인을 GPU로 이동합니다. LoRA 가중치를 고정된 사전 학습된 모델 가중치와 병합할 때 `scale` 매개변수를 사용하여 병합할 가중치의 양을 선택적으로 조정할 수 있습니다.

<Tip>

💡 `scale` 값이 `0`이면 LoRA 가중치를 사용하지 않고 기본 모델 가중치만 사용하는 것과 같고, `scale` 값이 `1`이면 완전히 미세 조정된 LoRA 가중치만 사용하는 것과 같습니다. `0`과 `1` 사이의 값은 두 가중치 사이를 보간합니다.

</Tip>

In [ ]:
pipe.unet.load_attn_procs(lora_model_path)
pipe.to("cuda")

# LoRA 미세 조정 모델의 가중치 절반과 기본 모델의 가중치 절반을 사용합니다.

In [ ]:
image = pipe(
    "A pokemon with blue eyes.", num_inference_steps=25, guidance_scale=7.5, cross_attention_kwargs={"scale": 0.5}
).images[0]

# 완전히 미세 조정된 LoRA 모델의 가중치를 사용합니다.

In [ ]:
image = pipe("A pokemon with blue eyes.", num_inference_steps=25, guidance_scale=7.5).images[0]
image.save("blue_pokemon.png")

<Tip>

Hub에서 LoRA 매개변수를 로드하고 Hub 리포지토리에 `base_model` 태그(예: [이것](https://huggingface.co/sayakpaul/sd-model-finetuned-lora-t4/blob/main/README.md?code=true#L4))가 있는 경우 다음을 수행할 수 있습니다.

In [ ]:
from huggingface_hub.repocard import RepoCard

lora_model_id = "sayakpaul/sd-model-finetuned-lora-t4"
card = RepoCard.load(lora_model_id)
base_model_id = card.data.to_dict()["base_model"]

pipe = StableDiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
...

</Tip>

## 드림부스

[DreamBooth](https://arxiv.org/abs/2208.12242)는 피사체의 몇 가지 이미지만으로 Stable Diffusion과 같은 텍스트-이미지 모델을 개인화하여 다양한 컨텍스트에서 피사체의 사실적인 이미지를 생성하는 미세 조정 기법입니다. 그러나 DreamBooth는 하이퍼파라미터에 매우 민감하며 과적합되기 쉽습니다. 고려해야 할 몇 가지 중요한 하이퍼파라미터에는 학습 시간(학습률, 학습 단계 수)과 추론 시간(단계 수, 스케줄러 유형)에 영향을 미치는 하이퍼파라미터가 포함됩니다.

<Tip>

💡 DreamBooth 실험 및 권장 설정에 대한 심층 분석은 [🧨 Diffusers를 사용한 DreamBooth로 Stable Diffusion 학습](https://huggingface.co/blog/dreambooth) 블로그를 참조하십시오.

</Tip>

### 학습[[dreambooth-training]]

DreamBooth와 LoRA를 사용하여 [`stable-diffusion-v1-5`](https://huggingface.co/runwayml/stable-diffusion-v1-5)를 일부 🐶 [개 이미지](https://drive.google.com/drive/folders/1BO_dyz-p65qhBRRMRA4TbZ8qW4rB99JZ)로 미세 조정해 보겠습니다. 이러한 이미지를 다운로드하여 디렉터리에 저장합니다. 자신의 데이터셋을 사용하려면 [학습용 데이터셋 만들기](https://huggingface.co/docs/diffusers/main/en/training/create_dataset) 가이드를 참조하십시오.

시작하려면 `MODEL_NAME` 환경 변수(Hub 모델 리포지토리 ID 또는 모델 가중치가 포함된 디렉터리 경로)를 지정하고 [`pretrained_model_name_or_path`](https://huggingface.co/docs/diffusers/en/api/diffusion_pipeline#diffusers.DiffusionPipeline.from_pretrained.pretrained_model_name_or_path) 인수에 전달합니다. 또한 이미지가 포함된 디렉터리 경로로 `INSTANCE_DIR`을 설정해야 합니다.

`OUTPUT_DIR` 변수는 선택 사항이며 Hub에 모델을 저장할 위치를 지정합니다.

```bash
export MODEL_NAME="runwayml/stable-diffusion-v1-5"
export INSTANCE_DIR="path-to-instance-images"
export OUTPUT_DIR="path-to-save-model"
```

학습을 시작하기 전에 알아야 할 몇 가지 플래그가 있습니다.

* `--push_to_hub`는 학습된 LoRA 임베딩을 Hub에 저장합니다.
* `--report_to=wandb`는 학습 결과를 Weights & Biases 대시보드에 보고하고 기록합니다(예시로 이 [보고서](https://wandb.ai/pcuenq/text2image-fine-tune/runs/b4k1w0tn?workspace=user-pcuenq)를 참조하십시오).
* `--learning_rate=1e-04`, LoRA를 사용하면 일반적으로 사용하는 것보다 높은 학습률을 사용할 수 있습니다.

이제 학습을 시작할 준비가 되었습니다(전체 학습 스크립트는 [여기](https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/train_dreambooth_lora.py)에서 찾을 수 있습니다). 스크립트는 모델 체크포인트와 `pytorch_lora_weights.bin` 파일을 리포지토리에 만들고 저장합니다.

또한 LoRA를 사용하여 텍스트 인코더를 추가로 미세 조정할 수도 있습니다. 대부분의 경우 약간의 컴퓨팅 증가로 더 나은 결과를 얻을 수 있습니다.
LoRA를 사용하여 텍스트 인코더 미세 조정을 허용하려면 `train_dreambooth_lora.py` 스크립트를 시작할 때 `--train_text_encoder`를 지정합니다.

```bash
accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="A photo of sks dog in a bucket" \
  --validation_epochs=50 \
  --seed="0" \
  --push_to_hub
```

### 추론[[dreambooth-inference]]

이제 [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)에서 기본 모델을 로드하여 모델을 추론에 사용할 수 있습니다.

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

model_base = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(model_base, torch_dtype=torch.float16)

미세 조정된 DreamBooth 모델에서 LoRA 가중치를 기본 모델 가중치 *위에* 로드한 다음 추론 속도를 높이기 위해 파이프라인을 GPU로 이동합니다. LoRA 가중치를 고정된 사전 학습된 모델 가중치와 병합할 때 `scale` 매개변수를 사용하여 병합할 가중치의 양을 선택적으로 조정할 수 있습니다.

<Tip>

💡 `scale` 값이 `0`이면 LoRA 가중치를 사용하지 않고 기본 모델 가중치만 사용하는 것과 같고, `scale` 값이 `1`이면 완전히 미세 조정된 LoRA 가중치만 사용하는 것과 같습니다. `0`과 `1` 사이의 값은 두 가중치 사이를 보간합니다.

</Tip>

In [ ]:
pipe.unet.load_attn_procs(lora_model_path)
pipe.to("cuda")

# LoRA 미세 조정 모델의 가중치 절반과 기본 모델의 가중치 절반을 사용합니다.

In [ ]:
image = pipe(
    "A picture of a sks dog in a bucket.",
    num_inference_steps=25,
    guidance_scale=7.5,
    cross_attention_kwargs={"scale": 0.5},
).images[0]

# 완전히 미세 조정된 LoRA 모델의 가중치를 사용합니다.

In [ ]:
image = pipe("A picture of a sks dog in a bucket.", num_inference_steps=25, guidance_scale=7.5).images[0]
image.save("bucket-dog.png")

학습 중에 `--train_text_encoder`를 사용한 경우 `pipe.load_lora_weights()`를 사용하여 LoRA 가중치를 로드합니다. 예를 들어 다음과 같습니다.

In [ ]:
from huggingface_hub.repocard import RepoCard
from diffusers import StableDiffusionPipeline
import torch

lora_model_id = "sayakpaul/dreambooth-text-encoder-test"
card = RepoCard.load(lora_model_id)
base_model_id = card.data.to_dict()["base_model"]

pipe = StableDiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.load_lora_weights(lora_model_id)
image = pipe("A picture of a sks dog in a bucket", num_inference_steps=25).images[0]

<Tip>

LoRA 매개변수에 UNet과 텍스트 인코더가 모두 포함된 경우 `cross_attention_kwargs={"scale": 0.5}`를 전달하면 UNet과 텍스트 인코더 모두에 `scale` 값이 적용됩니다.

</Tip>

LoRA 매개변수를 로드하는 데 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline.load_lora_weights)를 사용하는 것이 [load_attn_procs()](https://huggingface.co/docs/diffusers/main/en/api/loaders#diffusers.loaders.UNet2DConditionLoadersMixin.load_attn_procs)보다 선호됩니다. 이는 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline.load_lora_weights)가 다음 상황을 처리할 수 있기 때문입니다.

* UNet과 텍스트 인코더에 대한 별도의 식별자가 없는 LoRA 매개변수(예: [`"patrickvonplaten/lora_dreambooth_dog_example"`](https://huggingface.co/patrickvonplaten/lora_dreambooth_dog_example)). 따라서 다음과 같이 할 수 있습니다.

  ```py 
  pipe.load_lora_weights(lora_model_path)
  ```

* UNet과 텍스트 인코더에 대한 별도의 식별자가 있는 LoRA 매개변수(예: [`"sayakpaul/dreambooth"`](https://huggingface.co/sayakpaul/dreambooth)).

**참고** [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline.load_lora_weights)와 [load_attn_procs()](https://huggingface.co/docs/diffusers/main/en/api/loaders#diffusers.loaders.UNet2DConditionLoadersMixin.load_attn_procs) 모두에 로컬 디렉터리 경로를 제공할 수 있습니다. 지원되는 입력에 대해 알아보려면 각 docstring을 참조하십시오.

## Diffusers에서 A1111 테마 LoRA 체크포인트 지원

사용자에게 A1111과의 원활한 상호 운용성을 제공하기 위해 제한된 용량으로 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline.load_lora_weights)를 사용하여 A1111 형식의 LoRA 체크포인트를 로드하는 것을 지원합니다.
이 섹션에서는 Diffusers에서 [CivitAI](https://civitai.com/)의 A1111 형식 LoRA 체크포인트를 로드하고 이를 사용하여 추론을 수행하는 방법을 설명합니다.

먼저 체크포인트를 다운로드합니다. 데모용으로 [이것](https://civitai.com/models/13239/light-and-shadow)을 사용합니다.

```bash
wget https://civitai.com/api/download/models/15603 -O light_and_shadow.safetensors
```

다음으로 [~DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/diffusion_pipeline#diffusers.DiffusionPipeline)을 초기화합니다.

In [ ]:
import torch

from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

pipeline = StableDiffusionPipeline.from_pretrained(
    "gsdf/Counterfeit-V2.5", torch_dtype=torch.float16, safety_checker=None
).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
    pipeline.scheduler.config, use_karras_sigmas=True
)

그런 다음 CivitAI에서 다운로드한 체크포인트를 로드합니다.

In [ ]:
pipeline.load_lora_weights(".", weight_name="light_and_shadow.safetensors")

<Tip warning={true}>

`safetensors` 형식으로 체크포인트를 로드하는 경우 `safetensors`가 설치되어 있는지 확인하십시오.

</Tip>

그런 다음 추론을 실행할 차례입니다.

In [ ]:
prompt = "masterpiece, best quality, 1girl, at dusk"
negative_prompt = ("(low quality, worst quality:1.4), (bad anatomy), (inaccurate limb:1.2), "
                   "bad composition, inaccurate eyes, extra digit, fewer digits, (extra arms:1.2), large breasts")

images = pipeline(prompt=prompt, 
    negative_prompt=negative_prompt, 
    width=512, 
    height=768, 
    num_inference_steps=15, 
    num_images_per_prompt=4,
    generator=torch.manual_seed(0)
).images

아래는 LoRA와 비 LoRA 결과 간의 비교입니다.

![lora_non_lora](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lora_non_lora_comparison.png)

Hugging Face Hub에 유사한 체크포인트가 저장되어 있는 경우 다음과 같이 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline.load_lora_weights)를 사용하여 직접 로드할 수 있습니다.

In [ ]:
lora_model_id = "sayakpaul/civitai-light-shadow-lora"
lora_filename = "light_and_shadow.safetensors"
pipeline.load_lora_weights(lora_model_id, weight_name=lora_filename)